# Primer Intento de clasificador



Como el titulo lo dice, primer intento de clasificador usando como referencia el laboratorio x .
Se considera para este caso el archivo movies_final_imdb el cual contiene:
    
    movieId+nota_imbd_aprox+duracion+categoria+actores+directores
    
    
Nota : Falta agregar los directores y quizas las fechas para ve como cambia. 

In [39]:
## Librerias Basicas:
import csv
import numpy as np
import pandas as pd

#Metricas y Estadisticas:
from statsmodels.stats.weightstats import ttest_ind
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.utils import shuffle
from sklearn.cross_validation import train_test_split
from sklearn.metrics import classification_report
from sklearn import cross_validation


# Clasificadores : Tree, SVM, RandomForest, Regression, KNN.
from sklearn.svm import SVC  # support vector machine classifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB  # naive bayes
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn import linear_model
from sklearn.neighbors import KNeighborsClassifier
from sklearn.dummy import DummyClassifier



Cargamos el archivo 

In [40]:
#Cargamos el Archivo.

movies = pd.read_csv("movies_final.csv", header = 0)
score = pd.read_csv("movielens.csv", header = 0)

movies_header = list(movies.columns.values)
movies = movies.as_matrix()

score = np.ravel(score.as_matrix())
#score2 = score
#movies2 = movies
#movies = movies[0:4405]
#score = score[0:4405]

print (movies[0])

#score = np.asarray(score, dtype="|S6")
#print("Movies:\n", movies[4451])
#print("Score:\n", score[4451])
#print(movies_header)




[74  0  1 ...,  0  0  0]


# Reestructuracion del codigo y validación.

Definimos las funciones del clasificador:

In [48]:
#Variar el num_test modifica la probabilidad de estar mas cerca del promedio al hacer un t-test. 
#Creo que con 30 es suficiente para ese tipo de distribuciones.

def run_classifier(clf, X, y, num_tests=30):
    scores = []
    
    for i in range(num_tests):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.80)
        clf.fit(X_train, y_train)
        scores.append(clf.score(X_test, y_test))

    return np.array(scores)

Agregamos varios tipos de clasificadores :

In [49]:
c0 = ("Base", DummyClassifier(strategy='stratified'))
c1 = ("SVM", SVC(kernel='linear'))
c2 = ("DT", DecisionTreeClassifier())
c3 = ("NB", GaussianNB())
c4 = ("KNN", KNeighborsClassifier(n_neighbors=5))
c5 = ("RandomForest", RandomForestClassifier(n_estimators=10))

classifiers = [c0, c1, c2, c3, c4,c5,]
result_list = []

Corremos las pruebas para cada test. 

In [50]:
#Funcion Pesada, correr solo una vez. 

for name, clf in classifiers:
    accuracys = run_classifier(clf ,movies , score)
    result_list.append((name, accuracys))
    

In [51]:
#Impresion de los resultados

print("+ indica diferencia significativa\n")

for name1, results1 in result_list:
    print("Comparando %s - Accuracy: %.2f" % (name1, results1.mean()))
    for name2, results2 in result_list:
        if name1 == name2:
            continue

        _, p_value, __ = ttest_ind(results1, results2)
        
        if p_value <= 0.05:
            sig = "+"
        else:
            sig = ""
        print("%s:\t%.2f %s" % (name2, results2.mean(), sig))
    print()

+ indica diferencia significativa

Comparando Base - Accuracy: 0.45
SVM:	0.59 +
DT:	0.53 +
NB:	0.39 +
KNN:	0.54 +
RandomForest:	0.56 +
()
Comparando SVM - Accuracy: 0.59
Base:	0.45 +
DT:	0.53 +
NB:	0.39 +
KNN:	0.54 +
RandomForest:	0.56 +
()
Comparando DT - Accuracy: 0.53
Base:	0.45 +
SVM:	0.59 +
NB:	0.39 +
KNN:	0.54 
RandomForest:	0.56 +
()
Comparando NB - Accuracy: 0.39
Base:	0.45 +
SVM:	0.59 +
DT:	0.53 +
KNN:	0.54 +
RandomForest:	0.56 +
()
Comparando KNN - Accuracy: 0.54
Base:	0.45 +
SVM:	0.59 +
DT:	0.53 
NB:	0.39 +
RandomForest:	0.56 +
()
Comparando RandomForest - Accuracy: 0.56
Base:	0.45 +
SVM:	0.59 +
DT:	0.53 +
NB:	0.39 +
KNN:	0.54 +
()


# Predicciones de algunas peliculas

Usando la siguiente funcion, se puede imprimir mas facilmente los resultados y luego calcular algunas metricas importantes. 

In [77]:
sample_movies = [("Toy Story", movies[0], score[0]),("Avatar", movies[3], score[3])]

predicted_scores = []

clf_tst = classifiers[1]
print(clf_tst)
#print (sample_movies[0][1])

clf_tst.predict(movies[0])

#Corre con la ultima ejecucion del fit en la seccion anterior. ESTO SE PUEDE MEJORAR!
#for clf_name, clf in classifiers:
#    for sample_name, sample_data, sample_score in sample_movies:
#        clf.predict(sample_data)

('SVM', SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False))


AttributeError: 'tuple' object has no attribute 'predict'

# Parte de los clasificadores - Version Antigua.

# KNN basico



In [68]:
# Primer Clasificador : K-nearnest-neighbors

knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(movies, score)
score_pred = knn.predict(movies)
accuracy_score(score, score_pred)

print(classification_report(score,score_pred))

             precision    recall  f1-score   support

          0       0.00      0.00      0.00         5
          1       0.00      0.00      0.00        15
          2       0.51      0.29      0.37       459
          3       0.68      0.82      0.74      2618
          4       0.61      0.47      0.53      1352
          5       0.00      0.00      0.00         3

avg / total       0.64      0.65      0.63      4452



## KNN manual


In [69]:
# Primer Clasificador : K-nearnest-neighbors


X_train, X_test, y_train, y_test = cross_validation.train_test_split(movies, score, test_size=0.33)

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
accuracy_score(y_test, y_pred)

print(classification_report(y_test, y_pred))


             precision    recall  f1-score   support

          0       0.00      0.00      0.00         2
          1       0.00      0.00      0.00         4
          2       0.24      0.24      0.24       140
          3       0.62      0.70      0.66       858
          4       0.50      0.39      0.44       465
          5       0.00      0.00      0.00         1

avg / total       0.55      0.56      0.55      1470



## KNN - Cross Validation

In [70]:
knn_cross = cross_validation.cross_val_score(knn, movies, score, cv=3)

print knn_cross
print("Accuracy: %0.2f (+/- %0.2f)" % (knn_cross.mean(), knn_cross.std() * 2))




[ 0.53131313  0.54410774  0.51147099]
Accuracy: 0.53 (+/- 0.03)


Mejora al momento de reducir las clases

# Support Vector Machine - SVM

In [71]:

movies_svm = svm.SVC()
movies_svm.fit(movies, score)  

score_pred = movies_svm.predict(movies)
accuracy_score(score, score_pred)

print(classification_report(score,score_pred))


             precision    recall  f1-score   support

          0       0.00      0.00      0.00         5
          1       0.00      0.00      0.00        15
          2       0.00      0.00      0.00       459
          3       0.61      0.95      0.74      2618
          4       0.63      0.17      0.26      1352
          5       0.00      0.00      0.00         3

avg / total       0.55      0.61      0.52      4452



## SVM con Cross-Validation

In [74]:
svm_cross = svm.SVC()
cross = cross_validation.cross_val_score(svm_cross, movies, score, cv=10)

#Estadisticos respecto al cross validation
print cross
print("Accuracy: %0.2f (+/- %0.2f)" % (cross.mean(), cross.std() * 2))

#Res previos [ 0.61771058  0.60606061  0.60505415]
#Accuracy: 0.61 (+/- 0.01)

/home/pablo/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:516: Warning: The least populated class in y has only 3 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=10.
  % (min_labels, self.n_folds)), Warning)


[ 0.62276786  0.62053571  0.59284116  0.60538117  0.60089686  0.6036036
  0.59459459  0.6036036   0.61625282  0.61312217]
Accuracy: 0.61 (+/- 0.02)


## SVM test set manual

In [86]:
X_train, X_test, y_train, y_test = cross_validation.train_test_split(movies, score, test_size=0.33, random_state=45)


svm_manual = svm.SVC()
svm_manual.fit(X_train, y_train)
y_pred = svm_manual.predict(X_test)
accuracy_score(y_test, y_pred)
# test_size, random_state
# 0,33 , 37 -> 0.59863945578231292
# 0,33 , 45 -> 0.61360544217687074
# 0,33 , 80 -> 0.60612244897959189

# 0,66 , 37 -> 0.60598843143926506
# 0,66 , 45 -> 0.60428717250765562


0.61360544217687074

In [99]:
#predict toy_story
toy_story = movies[0]
print "\n\nNota original de toy-story : "
print score[0]
#print toy_story
print "Prediccion de nota para toy story:"
print svm_manual.predict([movies[0]])

print "Nota original de avatar"
print score[3111]
print "Prediccion par avatar"
print svm_manual.predict([movies[3111]])

print "Nota original Transformers 4"
print score[4057]
print "Prediccion para Transformers 4"
print svm_manual.predict([movies[4057]])

print "Nota Original Avatar: The last Airbender"
print score[3233]
print "Prediccion para Avatar : The last Airbender"
print svm_manual.predict([movies[3233]])

print "Nota original Justin Bieber: Never Say Never "
print score[4189]
print "Prediccion para Justin Bieber: Never Say Never"
print svm_manual.predict([movies[4189]])

print "Nota original Sex and the city 2"
print score[3214]
print "Prediccion para Sex and the city 2"
print svm_manual.predict([movies[4189]])

print "Nota original Titanic"
print score[458]
print "Prediccion para Titanic"
print svm_manual.predict([movies[458]])



Nota original de toy-story : 
4
Prediccion de nota para toy story:
[3]
Nota original de avatar
4
Prediccion par avatar
[4]
Nota original Transformers 4
3
Prediccion para Transformers 4
[4]
Nota Original Avatar: The last Airbender
2
Prediccion para Avatar : The last Airbender
[3]
Nota original Justin Bieber: Never Say Never 
1
Prediccion para Justin Bieber: Never Say Never
[3]
Nota original Sex and the city 2
2
Prediccion para Sex and the city 2
[3]
Nota original Titanic
3
Prediccion para Titanic
[4]


-No Acerto a toy-story
-Acertó a avatar
-No acerto a transformers :(


# Random Forest

## Random forest con cross validation.

In [103]:
#Random forest con cross-validation.

forest_cross = RandomForestClassifier(n_estimators=10)
forest_cross_score = cross_validation.cross_val_score(forest_cross, movies, score, cv=3)
print forest_cross_score
print("Accuracy: %0.2f (+/- %0.2f)" % (forest_cross_score.mean(), forest_cross_score.std() * 2))


# cv = 3, n = 10
# array([ 0.54456929,  0.57389347,  0.54954955])

# cv = 3, n = 10
# array([ 0.55735661,  0.57927591,  0.59699625,  0.55319149,  0.57446809])



[ 0.54478114  0.56969697  0.5802969 ]
Accuracy: 0.56 (+/- 0.03)


## Random Forest test set Manual

In [82]:
X_train, X_test, y_train, y_test = cross_validation.train_test_split(movies, score, test_size=0.3)
#, random_state=37

forest_manual = RandomForestClassifier(n_estimators=10)
forest_manual.fit(X_train, y_train)
y_pred = forest_manual.predict(X_test)
accuracy_score(y_test, y_pred)


# test_size = 0.33
# 0.57166666666666666

# test_size = 0.5
# 0.55449999999999999


0.57211538461538458

# Arbol de Decision

In [83]:
movies_tree = tree.DecisionTreeClassifier()
cross_validation.cross_val_score(movies_tree, movies, score, cv=10)


/home/pablo/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:516: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=10.
  % (min_labels, self.n_folds)), Warning)


array([ 0.54415274,  0.53349282,  0.51079137,  0.55395683,  0.51318945,
        0.5373494 ,  0.55903614,  0.56280193,  0.56763285,  0.5410628 ])

Aparentemente, tanto el arbol de decision como random forest tree no tienen un rendimiento mejor que los otros. ¿Por qué?

# Dummy Classifier

Que debemos hacer ahora ? -> Validar los modelos comparandolos con un dummy 